<a href="https://colab.research.google.com/github/theshivamgiri/Smart_Irrigation_System-AquaSmartAgri/blob/main/UI_for_farmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np

# URL for ThingSpeak channel data (replace 'YOUR_CHANNEL_ID' and 'YOUR_API_KEY')
url = 'https://api.thingspeak.com/channels/2648719/feeds.json?api_key=T9BTHEE3NKREFMUD&results=1000'

response = requests.get(url)
data = response.json()

# Convert JSON data to a DataFrame
feeds = data['feeds']
df = pd.DataFrame(feeds)
df['created_at'] = pd.to_datetime(df['created_at'])
df['field1'] = pd.to_numeric(df['field1'], errors='coerce')  # Temperature
df['field2'] = pd.to_numeric(df['field2'], errors='coerce')  # Humidity

print(df.tail())

                  created_at  entry_id  field1  field2
58 2024-11-15 11:25:38+00:00        59    -3.8    72.0
59 2024-11-15 11:28:06+00:00        60    -3.8    72.0
60 2024-11-15 11:28:28+00:00        61    22.9    72.0
61 2024-11-15 11:29:39+00:00        62    22.9    72.0
62 2024-11-15 14:40:56+00:00        63    42.2    40.0


In [ ]:
# Drop rows with NaN values
df.dropna(subset=['field1', 'field2'], inplace=True)

# Rename columns for clarity
df.rename(columns={'field1': 'Temperature', 'field2': 'Humidity'}, inplace=True)

# Add features such as time of day or average temperature/humidity for better modeling
df['hour'] = df['created_at'].dt.hour

# Example target variable: suggest whether irrigation is needed
df['Irrigation_Need'] = np.where((df['Temperature'] > 30) & (df['Humidity'] < 50), 1, 0)  # Simplified rule

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data for training
X = df[['Temperature', 'Humidity', 'hour']]
y = df['Irrigation_Need']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

# # Function for predicting irrigation need
# def predict_irrigation(temp, humidity, hour):
#     prediction = model.predict([[temp, humidity, hour]])
#     return "Irrigation Needed" if prediction[0] == 1 else "No Irrigation Needed"


# Output should be in Hindi
# Updated function to include feature names
def predict_irrigation(temp, humidity, hour):
    input_data = pd.DataFrame([[temp, humidity, hour]], columns=['Temperature', 'Humidity', 'hour'])
    prediction = model.predict(input_data)
    return "सिंचाई की आवश्यकता है" if prediction[0] == 1 else "सिंचाई की आवश्यकता नहीं है"

#example - irrigation needed @(90,9,1), , not needed @(9,20,10)
# Test the function
print(predict_irrigation(54,2,1))

Accuracy: 0.9166666666666666
सिंचाई की आवश्यकता है


In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
!pip install streamlit
!streamlit run app.py & npx localtunnel --port 8501

import streamlit as st
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load the pre-trained model (assuming you have saved it)
# Use `pickle.dump(model, open('model.pkl', 'wb'))` after training to create this file
model_filename = 'model.pkl'
try:
    model = pickle.load(open(model_filename, 'rb'))
except FileNotFoundError:
    st.error("Model file not found. Please train the model and save it as 'model.pkl'.")

# Function to make predictions
def predict_irrigation(temp, humidity, hour):
    prediction = model.predict([[temp, humidity, hour]])
    return "सिंचाई की आवश्यकता है" if prediction[0] == 1 else "सिंचाई की आवश्यकता नहीं है"

# Streamlit UI setup
st.set_page_config(page_title="स्मार्ट सिंचाई प्रणाली", layout="centered")

st.title("स्मार्ट सिंचाई सहायता प्रणाली")
st.subheader("अपनी फसल के लिए सिंचाई की ज़रूरत का पूर्वानुमान")

# Input fields for farmers (in Hindi)
temp = st.number_input("तापमान दर्ज करें (°C)", min_value=-10.0, max_value=50.0, step=0.1)
humidity = st.number_input("नमी दर्ज करें (%)", min_value=0.0, max_value=100.0, step=0.1)
hour = st.slider("समय (घंटे में)", 0, 23, 12)  # Default current time input

# Button to get prediction
if st.button("परिणाम देखें"):
    if 'model' in locals():
        result = predict_irrigation(temp, humidity, hour)
        st.success(f"सुझाव: {result}")
    else:
        st.warning("कृपया पहले मॉडल को लोड करें।")

# Tips for usage (optional)
st.info("टिप: सिंचाई की आवश्यकता वाले परिणाम का मतलब है कि आपके खेत को पानी की आवश्यकता है।")

# Footer (optional)
st.write("**यह ऐप किसानों की सहायता के लिए डिज़ाइन किया गया है।**")

In [ ]:
from IPython.core.display import display, HTML

html_code = """
<!DOCTYPE html>
<html lang="hi">
<head>
    <meta charset="UTF-8">
    <title>स्मार्ट सिंचाई प्रणाली</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <h1>स्मार्ट सिंचाई सहायता प्रणाली</h1>
    <form id="irrigationForm">
        <label for="temp">तापमान (°C):</label>
        <input type="number" id="temp" required>
        <label for="humidity">नमी (%):</label>
        <input type="number" id="humidity" required>
        <label for="hour">समय (घंटे):</label>
        <input type="number" id="hour" required>
        <button type="submit">परिणाम देखें</button>
    </form>
    <p id="result"></p>

    <script>
        document.getElementById('irrigationForm').onsubmit = async (e) => {
            e.preventDefault();
            const temp = document.getElementById('temp').value;
            const humidity = document.getElementById('humidity').value;
            const hour = document.getElementById('hour').value;
            const response = await fetch('/predict', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ Temperature: temp, Humidity: humidity, Hour: hour })
            });
            const data = await response.json();
            document.getElementById('result').innerText = `सुझाव: ${data.result}`;
        };
    </script>
</body>
</html>

"""
display(HTML(html_code))

# Output should be in Hindi
# Updated function to include feature names
import time
def predict_irrigation(temp, humidity, hour, delay=18):
    input_data = pd.DataFrame([[temp, humidity, hour]], columns=['Temperature', 'Humidity', 'hour'])
    prediction = model.predict(input_data)

    # Introduce delay
    time.sleep(delay)

    return "सिंचाई की आवश्यकता है" if prediction[0] == 1 else "सिंचाई की आवश्यकता नहीं है"

# Test the function with delay
print(predict_irrigation(54, 200, 1, delay=18))

सिंचाई की आवश्यकता नहीं है
